In [1]:
import requests
import json
import pandas as pd

In [2]:
def get_json(ticker):
    output = {}
    output['ticker'] = ticker

    quote = requests.get('https://finnhub.io/api/v1/quote?symbol='+ticker+'&token=brqivm7rh5rc4v2pmq8g')
    quote_json = json.loads(str(quote.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    output['quote'] = quote_json
        
    aggr = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol='+ticker+'&resolution=D&token=bt6pumf48v6oqmgq7j1g')
    aggr_json = json.loads(str(aggr.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    output['technicalAnalysis'] = aggr_json

    senti = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol='+ticker+'&token=bt6pv3n48v6oqmgq7j70')
    senti_json = json.loads(str(senti.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    output['sentiment'] = senti_json
        
    fins = requests.get('https://finnhub.io/api/v1/stock/metric?symbol='+ticker+'&metric=all&token=bt4oq4f48v6um6kgq5gg')
    fins_json = json.loads(str(fins.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    fins_json = fins_json["metric"]
    output['financials'] = fins_json

    return json.dumps(output, indent=2)

In [3]:
def get_quote(ticker):
    quote = get_json(ticker)
    quote_json = eval(quote)
    
    close = quote_json['quote']['c']
    prevClose = quote_json.get('quote').get('pc')
    change = round((close/prevClose-1)*100, 2)
    signal = quote_json.get('technicalAnalysis').get('technicalAnalysis').get('signal')
    articlesLastWk = quote_json.get('sentiment').get('buzz').get('articlesInLastWeek')
    weeklyAvg = int(quote_json.get('sentiment').get('buzz').get('weeklyAverage')+0.5)
    newsScore = quote_json.get('sentiment').get('companyNewsScore')
    sectorAvgNewsScore = quote_json.get('sentiment').get('sectorAverageNewsScore')
    buzz = quote_json.get('sentiment').get('buzz').get('buzz')
    bullishPercent = round(quote_json.get('sentiment').get('sentiment').get('bullishPercent')*100, 2)
    sectorAvgBullishPercent = round(quote_json.get('sentiment').get('sectorAverageBullishPercent')*100, 2)
    
    return {'ticker':ticker, 'change':change,'close':close,'prevClose':prevClose,'signal':signal,'articlesLastWk':articlesLastWk,'weeklyAvg':weeklyAvg,'newsScore':newsScore,'sectorAvgNewsScore':sectorAvgNewsScore,'buzz':buzz,'bullishPercent':bullishPercent,'sectorAvgBullishPercent':sectorAvgBullishPercent}

In [4]:
df = pd.DataFrame(columns=['ticker','change','close','prevClose','signal','articlesLastWk','weeklyAvg','newsScore','sectorAvgNewsScore','buzz','bullishPercent','sectorAvgBullishPercent'])

for ticker in ['JPM', 'IBM', 'UNH', 'PG', 'BA', 'VZ', 'GS', 'INTC', 'HD', 'MCD', 'CSCO', 'V', 'MSFT', 'KO', 'AXP', 'AMGN', 'MRK', 'CVX', 'MMM', 'TRV', 'JNJ', 'DIS', 'HON', 'CAT', 'NKE', 'CRM', 'WBA', 'AAPL', 'DOW', 'WMT']:
    df = df.append(get_quote(ticker), ignore_index = True)
    
df = df.sort_values(by=['bullishPercent','newsScore','articlesLastWk','change'], ascending=False)
df = df.reset_index(drop=True)
df.head(len(df))

,ticker,change,close,prevClose,signal,articlesLastWk,weeklyAvg,newsScore,sectorAvgNewsScore,buzz,bullishPercent,sectorAvgBullishPercent
0,PG,0.66,142.38,141.45,neutral,87,42,1.0000,0.5309,2.0591,100.00,62.66
1,HD,0.65,283.00,281.16,buy,18,20,0.9166,0.5309,0.9000,100.00,62.66
2,CSCO,0.00,38.82,38.82,neutral,15,20,0.7946,0.5222,0.7692,100.00,64.46
3,CAT,-0.63,168.59,169.66,buy,13,18,0.7706,0.5073,0.7222,100.00,59.60
4,UNH,1.50,330.60,325.73,buy,17,27,0.7329,0.5029,0.6355,100.00,58.72
5,CRM,0.34,250.52,249.67,buy,20,23,0.7165,0.5222,0.8695,100.00,64.46
6,AMGN,-0.40,227.06,227.98,sell,11,14,0.6964,0.5029,0.8000,100.00,58.72
7,NKE,-0.02,129.99,130.02,neutral,20,23,0.6804,0.5309,0.8602,100.00,62.66
8,HON,-0.74,175.54,176.85,buy,9,16,0.6473,0.5073,0.5714,100.00,59.60
9,KO,-0.32,50.52,50.68,neutral,79,41,0.8695,0.5309,1.9151,90.90,62.66
